CNN은 이미지 인식과 음성 인식 등 다양한 곳에서 사용된다. 특히 이미지인식분야에서 딥러닝을 활용한 기법은 거의 다 CNN을 기초로 한다.  

# 7.1전체 구조  
합성곱계층과 풀링계층이 새롭게 등장  
완전연결: 신경망은 인접하는 계층의 모든 뉴런과 결합되어있다.완전히 연결된 계층을 Affine계층이라는 이름으로 구현  

### 완전연결신경망  
- Affine계층 뒤에 활성화 함수를 갖는 ReLU계층이 이어진다.    
구조)Affine-ReLU조합이 4개 쌓여있고 마지막 5번째층은 Affine계층에 이어 소프트맥스 계층에서 최종결과(확률)을 출력  

### CNN으로 이뤄진 네트워크  
- cnn에서는 새로운 '합성곱 계층'과 '풀링계층'이 추가된다.  
- conv-relu-(pooling)흐름으로 연결된다.(풀링계층은 생략하기도 함)  
구조)conv-relu흐름연결+마지막 출력계층에서는 'Affine-Softmax'조합 그대로 사용  

# 7.2 합성곱 계층  
- CNN에서는 패딩, 스트라이드의 고유 용어 등장  
- 각 계층 사이에 3차원 데이터같이 입체적인 데이터가 흐른다는 점에서 완전연결신경망과 다름  

## 7.2.1 완전연결계층의 문제점  
완전연결계층에서는 인접하는 계층의 뉴런이 모두 연결되고 출력의 수는 임의로 정할 수 있다.  
완전연결계층의 문제점:데이터의 형상이 무시됨 (ex.이미지는 3차원 데이터인데 이것이 1차원데이터로 평탄화된다)  
- 이미지는 3차원 형상인데 공간적으로 가까운 픽셀은 값이 비슷  
- rgb의 각 채널은 서로 밀접하게 관련되거나, 거리가 먼 픽셀끼리는 별 연관이 없는 등 3차원 속에서 의미를 갖는 본질적인 패턴이 숨어있을 것이다.  


But,완전연결계층은 형상을 무시하고 모든 입력 데이터를 동등한 뉴런으로 취급하여 형상에 담긴 정보를 살릴 수 없다  

특징맵: 합성곱 계층의 입출력 데이터  
입력특징맵: 합성곱 계층의 입력데이터  
출력특징맵:합성곱 계층의 출력데이터  
cf)입출력 데이터와 '특징맵'을 같은 의미로 사용한다고 하자.  

## 7.2.2 합성곱 연산  
합성곱 연산:이미지처리에서 말하는 필터연산에 해당  
입력데이터는 세로,가로 방향의 형상을 가졌고, 필터 역시 세로,가로 방향의 차원을 갖는다.  
데이터와 필터 형상을 (높이, 너비)로 표기하며 필터를 '채널'이라고도 부른다  

- 합성곱 연산은 필터의 원도우를 일정간격으로 이동해가며 입력데이터에 적용한다.  
1. 입력과 필터에 대응하는 원소끼리 곱한후 그 총합을 구한다.(단일곱셈 누산)  
2. 결과를 해당장소에 저장한다.  

cf) 합성곱연산의 편향: 필터를 적용한 원소에 고정값(편향)을 더한다.이때 편향은 하나의 값만 존재!!!

## 7.2.3 패딩  
패딩:합성곱 연산을 수행하기 전에 입력데이터 주변을 특정값으로 채우기도 한다  
패딩은 출력 크기를 조정할 목적으로 사용한다.  
예를 들어, (4,4)입력데이터에 (3,3)필터를 적용하면 출력 (2,2)가 되어 입력보다 2만큼 줄어든다.  
이는 합성곱 연산을 몇번이나 되풀이하는 심층 신경망에서 문제요소임->출력크기가 어느 시점에서는 1이 되니까.  
즉, 입력데이터의 공간적 크기를 고정한 채 다음 계층에 전달할 수 있음  

## 7.2.4 스트라이드  
스트라이드:필터를 적용하는 위치의 간격  
ex)스트라이드를 2로 하면 필터를 적용하는 윈도우가 2칸씩 이동한다!  

패딩, 스트라이드, 출력크기의 관계 수식화  
입력크기(H,W) / 필터크기(FH, FW) / 출력크기(OH, OW) / 패딩 P / 스트라이드 S  
OH=(H+2P-FH)/S+1  
OW=(W+2P-FW)/S+1  
수식들은 정수로 나눠떨어지는 값이어야 함  

## 7.2.5 3차원 데이터의 합성곱 연산  
2차원일때 길이방향으로 특징 맵이 늘어났다.  
채널쪽으로 특징맵이 여러개 있다면, 입력데이터와 필터의 합성곱 연산을 채널마다 수행하고 결과를 더해서 하나의 출력을 얻는다.  
주의할점: 입력데이터의 채널수와 필터의 채널수가 같아야한다. 한편, 필터자체의 크기는 원하는 값으로 설정할 수 있다.(단, 모든 채널의 필터가 같은 크기여야 한다.)  

## 7.2.6 블록으로 생각하기  
3차원 합성곱 연산은 데이터와 필터를 직육면체 블록이라 생각하자.  
이때, 3차원 데이터를 다차원 배열로 나타낼때는 (채널, 높이, 너비)순서로 작성=(C,H,W)  
- (C=channel, H=height, W=weight)  

필터의 예시: 채널 수C,필터높이 FH,필터너비 FW의 경우 (C,FH,FW)사용  


합성곱연산에서 필터의 수 고려  
필터의 가중치데이터는 4차원 데이터이며 (출력 채널의 수, 입력채널의 수, 높이, 너비)순으로 쓴다  
ex)채널수3,크기 5x5인 필터가 20개= (20,3,5,5)  

### 합성곱 연산의 처리흐름  
입력데이터*필터->결과+편향->출력데이터  
편향은 채널하나에 값 하나씩 구성됨  
편향의 형상은 (FN,1,1)이고 필터의 출력 결과의 형상은 (FN,OH,OW)이다.  
편향의 각 값이 필터의 출력인 (FN,OH,OW)의 대응채널의 원소 전부에 더해진다  

cf)형상shape이 다른 블록의 덧셈도 가능(broadcasting사용)  

## 7.2.7 배치 처리  
합성곱 연산에서 배치처리를 지원하고자 함->각 계층을 흐르는 데이터의 차원을 늘려 4차원 데이터로 저장합니다. 구체적으로 데이터를 (데이터수, 채널수, 높이, 너비)순으로 저장한다  

합성곱연산의 처리흐름(배치처리):입력데이터*필터->N개의 데이터+편향->출력데이터  
이때,(각 데이터의 선두에 배치용차원을 추가)신경망에 4차원 데이터가 하나 흐를 때마다 데이터 N개에 대한 합성곱 연산이 이뤄짐.

## 7.3. 풀링 계층  
pooling:세로*가로 방향의 공간을 줄이는 연산  
- 풀링의 윈도우 크기와 스트라이드는 같은 값으로 설정하는 것이 보통  
최대풀링(max pooling):최댓값을 구하는 연산으로 '2x2'는 대상영역의 크기를 뜻함  
ex)2x2 최대 풀링은 2x2영역에서 가장 큰 원소를 하나 꺼낸다.  
    또 스트라이드는 2로 설정했고 2x2윈도우가 원소 2칸간격으로 이동한다.  

## 7.3.1 풀링계층의 특징  
- 학습해야할 매개변수가 없다  
- 채널수가 변하지 않는다  
- 입력의 변화에 영향을 적게 받는다  

# 7.4 합성곱/풀링 계층 구현하기  

## 7.4.1 4차원 배열  
CNN에서 계층 사이를 흐르는 데이터는 4차원데이터  
(10,1,28,28)은 높이 28, 너비 28 채널1개인 데이터가 10개라는 의미  
cf) 데이터 수, 채널 수 높이, 너비 순

In [1]:
import numpy as np
x=np.random.rand(10,1,28,28)
x.shape

(10, 1, 28, 28)

In [2]:
print(x[0].shape)#입력데이터의 첫번째 데이터에 접근
print(x[1].shape)#입력데이터의 두번째 데이터에 접근

(1, 28, 28)
(1, 28, 28)


In [3]:
x[0,0]#x[0][0]이랑 동치(첫번째 데이터의 첫 채널의 공간데이터에 접근)

array([[0.89584695, 0.86182708, 0.99464805, 0.67350613, 0.8829993 ,
        0.81184302, 0.92408522, 0.42433783, 0.91403156, 0.94099789,
        0.52833127, 0.37396337, 0.59304917, 0.1997283 , 0.30217886,
        0.39130051, 0.99090729, 0.45361502, 0.53572685, 0.04913771,
        0.22468356, 0.63485674, 0.77817666, 0.34066857, 0.69118834,
        0.51798168, 0.51686057, 0.25759794],
       [0.39703231, 0.15238229, 0.31642625, 0.38707098, 0.13532434,
        0.45483219, 0.07114615, 0.71599907, 0.95948724, 0.40281819,
        0.95042713, 0.78277675, 0.41189631, 0.78420514, 0.00463561,
        0.66446606, 0.78072997, 0.92312981, 0.99578631, 0.02092041,
        0.96759495, 0.39484899, 0.33192925, 0.20423549, 0.04476077,
        0.69038932, 0.28438438, 0.14040169],
       [0.8227638 , 0.27312372, 0.20418689, 0.29605669, 0.06449057,
        0.67487544, 0.80472012, 0.24396866, 0.94797546, 0.37047132,
        0.67491873, 0.61657949, 0.19873161, 0.91290153, 0.90850532,
        0.42229736, 0.7579

## 7.4.2 im2col로 데이터 전개하기  
im2col은 입력데이터를 필터링하기 좋게 전개하는 함수이다.  
3차원 입력데이터에 im2col을 적용하면 2차원 행렬로 바뀐다.(배치 안의 데이터 수까지 포함한 4차원 데이터를 2차원으로 변환한다.)  

실제 상황에서는 필터 적용 영역이 겹치게 되면, im2col로 전개한 후의 원소수가 원래 블록의 원소수보다 많아짐  
단점:im2col을 사용해 구현시, 메모리 많이 소비  

합성곱연산의 필터 처리 상세과정:필터를 세로로 1열 전개->im2col이 전개한 데이터와 행렬곱 계산->출력데이터를 reshaping한다  

im2col의 인터페이스  
im2col(input_data, filter_h, filter_w, stride=1, pad=0)  
input_data : (데이터 수, 채널 수, 높이, 너비)의 입력데이터  
filter_h : 필터의 높이  
filter_w : 필터의 너비  
stride : 스트라이드  
pad : 패딩  

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import sys,os
sys.path.append("/content/drive/MyDrive/deeplearning/deep-learning-from-scratch-master")
#from common.util import im2col

def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

x1=np.random.rand(1,3,7,7)
col1=np.random.rand(1,3,7,7)#데이터수,채널수,높이,너비
col1=im2col(x1,5,5,stride=1,pad=0)
print(col1.shape)
x2=np.random.rand(10,3,7,7)
col2=im2col(x2,5,5,stride=1,pad=0)
print(col2.shape)

class Convolution:
  def __init__(self,W,b,stride=1,pad=0):
    self.W=W
    self.b=b
    self.stride=stride
    self.pad=pad
  def forward(self,x):
    FN,C,FH,FW=self.W.shape
    N,C,H,W=x.shape
    out_h=int(1+(H+2*self.pad-FH)/self.stride)
    out_w=int(1+(W+2*self.pad-FW)/self.stride)
    col=im2col(x,FH,FW,self.stride,self.pad)
    col_W=self.W.reshape(FN,-1).T#필터 전개
    out=np.dot(col,col_W)+self.b
    out=out.reshape(N,out_h,out_w,-1).transpose(0,3,1,2)#(N,H,W,C)->(N,C,H,W)즉 데이터수,채널수,높이,너비
    return out

(9, 75)
(90, 75)


## 7.4.4 풀링계층 구현하기  
풀링계층 구현도 합성곱계층과 마찬가지로 im2col을 사용해 입력데이터를 전개한다  
전개한 행렬에서 행별 최댓값을 구한뒤 적절한 shape으로 성형  

In [8]:
class Pooling:
  def __init__(self,pool_h,pool_w,stride=1,pad=0):
    self.pool_h=pool_h
    self.pool_w=pool_w
    self.stride=stride
    self.pad=pad
  def forward(self,x):
    N,C,H,W=x.shape
    out_h=int(1+(H-self.pool_h)/self.stride)
    out_w=int(1+(W-self.pool_w)/self.stride)
    col=im2col(x,self.pool_h,self.pool_w,self.stride,self.pad)
    col=col.reshape(-1,self.pool_h*self.pool_w)
    out=np.max(col,axis=1)
    out=out.reshape(N,out_h,out_w,C).transpose(0,3,1,2)
    return out

#2차원 배열 행렬이라면 axis=0은 열방향,axis=1 행방향
#풀링계층 구현:1.입력데이터를 전개한다. 2.행별 최댓값을 구한다. 3.적절한 모양으로 성형한다.

# 7.5 CNN구현하기  
CNN네트워크:conv-relu-pooling-affine-relue-affine-softmax순으로 흐른다  

In [9]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import pickle
import numpy as np
from collections import OrderedDict
class SimpleConvNet:
    #input_size : 입력 크기（MNIST의 경우엔 784）
    #hidden_size_list : 각 은닉층의 뉴런 수를 담은 리스트（e.g. [100, 100, 100]）
    #output_size : 출력 크기（MNIST의 경우엔 10）
    #activation : 활성화 함수 - 'relu' 혹은 'sigmoid'
    #weight_init_std : 가중치의 표준편차 지정
    def __init__(self, input_dim=(1, 28, 28),
                conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
      filter_num = conv_param['filter_num']
      filter_size = conv_param['filter_size']
      filter_pad = conv_param['pad']
      filter_stride = conv_param['stride']
      input_size = input_dim[1]
      conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
      pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
      #초기화 인수로 주어진 합성곱 계층의 하이퍼 파리미터를 딕셔너리에서 꺼낸다. 합성곱계층의 출력 크기를 계산한다

      #가중치 초기화
      self.params = {}
      self.params['W1'] = weight_init_std * \
                          np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
      self.params['b1'] = np.zeros(filter_num)
      self.params['W2'] = weight_init_std * \
                          np.random.randn(pool_output_size, hidden_size)
      self.params['b2'] = np.zeros(hidden_size)
      self.params['W3'] = weight_init_std * \
                          np.random.randn(hidden_size, output_size)
      self.params['b3'] = np.zeros(output_size)

      #계층 생성
      self.layers=OrderedDict()
      self.layers['Conv1']=Convolution(self.params['W1'],self.params['b1'],conv_param['stride'],conv_param['pad'])
      self.layers['Relu1']=Relu()
      self.layers['Pool1']=Pooling(pool_h=2,pool_w=2,stride=2)#높이,너비,스트라이드,패딩0처리
      self.layers['Affine1']=Affine(self.params['W2'],self.params['b2'])
      self.layers['Relu2']=Relu()
      self.layers['Affine2']=Affine(self.params['W3'],self.params['b3'])
      self.last_layer=SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)
        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])
        return grads

    def gradient(self, x, t):#오차역전파법으로 기울기를 구하는
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

# 7.6 CNN시각화하기  

## 7.6.1 1번째 층의 가중치 시각화하기  
학습 전 필터는 무작위로 초기화되고 있어 흑백의 정도에 규칙성이 없음  
학습을 마친 필터는 규칙성 있는 이미지  
규칙성있는 필터는 에지(색상이 바뀐 경계선)와 볼륨(국소적으로 덩어리진 영역)를 보고있음  
합성곱 계층의 필터는 에지나 블롭 등의 원시적인 정보 추출가능, 뒤 계층에 전달  

## 7.6.2 층 깊이에 따른 추출 정보 변화  
1번째 층의 합성곱계층에서는 에지, 블롭등의 저수준 정보가 추출된다  
CNN의 각 계층에서는 계층이 깊어질수록 추출되는 정보가 더 추상화된다  
ex)처음 층은 에지에 반응하고, 이어서 텍스처, 더 복잡한 사물의 일부에 반응...층이 깊어지면서 뉴런 반응대상이 고급정보로 변화  

# 7.7 대표적인 CNN  

## 7.7.1 LeNet  
손글씨 숫자인식하는 네트워크  
합성곱계층과 풀링계층을 반복하고 마지막으로 완전연결계층을 거치면서 결과를 출력한다  

### LeNet과 Cnn의 비교(차이):
LeNet:  
- 활성화함수:시그모이드  
- 서브샘플링하여 중간데이터 크기 줄인다

CNN:
- 활성화함수:ReLU  
- 최대풀링이 주류  

## 7.7.2 AlexNet  
합성곱계층과 풀링계층을 거듭하면서 마지막으로 완전연결계층을 거쳐 결과 출력  
차이점:  
- 활성화함수:ReLU  
- LRN국소적 정규화실시하는 계층 이용  
- 드롭아웃을 사용  